<a href="https://colab.research.google.com/github/joshiaarav09/Customer_Review_Determiner/blob/main/Customer_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

class Sentiment:
  Negative = 'Negative'
  Neutral = 'Neutral'
  Positive = 'Positive'

class Review:
  def __init__(self ,text ,score):
    self.text = text
    self.score = score
    self.sentiment =self.get_sentiment()

  def get_sentiment(self):
    if self.score <=2:
      return Sentiment.Negative
    elif self.score == 3:
      return Sentiment.Neutral
    else:
      return Sentiment.Positive
class ReviewContainer:
  def __init__(self, reviews):
    self.reviews = reviews

  def get_text(self):
    return[x.text for x in self.reviews]

  def get_sentiment(self):
    return[x.sentiment for x in self.reviews]


  def evenly_distribute(self):
    negative = list(filter(lambda x: x.sentiment ==Sentiment.Negative, self.reviews))
    positive = list(filter(lambda x: x.sentiment ==Sentiment.Positive, self.reviews))
    positive_shrunk = positive[:len(negative)]
    self.reviews = negative + positive_shrunk
    random.shuffle(self.reviews)







In [ ]:
import json
file_name = '/content/Books_small_10000.json'
reviews = []
with open(file_name) as f:
  for line in f:
    review = json.loads(line)
    reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text


'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

# **Prep** **Data**

In [ ]:
from sklearn.model_selection import train_test_split
training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)


In [ ]:
train_container.evenly_distribute()

train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

# print(train_y.count(Sentiment.Positive))
# print(train_y.count(Sentiment.Negative))


# **Bag of words vectorization**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer()

# train_x_vectors = vectorizer.fit_transform(train_x)
vectorizer.fit(train_x)
train_x_vectors = vectorizer.transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())






I found this book to be very interesting. Started and didn't want to put it down. Enjoyed it very much. Good ending.
[[0. 0. 0. ... 0. 0. 0.]]


# **Classification**

# *Linear SVM*

In [ ]:
from sklearn import svm

clf_svm  = svm.SVC(kernel = 'linear')
clf_svm.fit(train_x_vectors, train_y)

clf_svm.predict(test_x_vectors[0])


array(['Negative'], dtype='<U8')

# *Decision Tree*


In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf_dec = DecisionTreeClassifier()

clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])


array(['Positive'], dtype='<U8')

# *Naive Bayes (Gaussian)*

In [ ]:
# from sklearn.naive_bayes import GaussianNB

# clf_gnb = GaussianNB()

# clf_gnb.fit(train_x_vectors, train_y)

# test[0]
# clf_gnb.predict(test_x_vectors[0])



TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

# *Logistic Regression*


In [ ]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()

clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])


array(['Negative'], dtype='<U8')

# **Evaluation**

In [ ]:
#Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))

0.8076923076923077
0.6442307692307693
0.8052884615384616


In [ ]:
#F1 Score
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors), average = None, labels=[Sentiment.Positive, Sentiment.Neutral, Sentiment.Negative])
f1_score(test_y, clf_dec.predict(test_x_vectors), average = None, labels=[Sentiment.Positive, Sentiment.Neutral, Sentiment.Negative])


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


array([0.64423077, 0.        , 0.64423077])

# **Tuning our model (with Grid Search)**

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C': (1,4,8,16,32)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [ ]:
print(clf_svm.score(test_x_vectors, test_y))


0.8076923076923077


# **Saving Model**

In [ ]:
import pickle

with open('/content/drive/MyDrive/Sentiment_classifier.pkl', 'wb') as f:
  pickle.dump(clf, f)

# **Loading Model**

In [ ]:
with open('/content/drive/MyDrive/Sentiment_classifier.pkl', 'rb') as f:
  loaded_clf= pickle.load(f)
